In [9]:
%matplotlib inline
import os
import numpy
from ipywidgets import interact, fixed

from supertomo.ui import arguments                     # Command line arguments
from supertomo.io import image_data                    # Data structure
from supertomo.reconstruction import fusion            # Image registration functions
from supertomo.ui import show
from supertomo.utils import itkutils


In [10]:
fuse_args = ("hela_full_test.hdf5 --dir=/home/sami/Data/SuperTomo2/Import "  
             "--max-nof-iterations=2  --first-estimate=constant " 
             "--fusion-method=summative --scale=40 --blocks=1 --pad=0").split()
            
print fuse_args

options = arguments.get_fusion_script_options(fuse_args)




['hela_full_test.hdf5', '--dir=/home/sami/Data/SuperTomo2/Import', '--max-nof-iterations=2', '--first-estimate=constant', '--fusion-method=summative', '--scale=40', '--blocks=1', '--pad=0']


In [11]:
full_path = os.path.join(options.working_directory,
                         options.data_file)

if not os.path.isfile(full_path):
    raise AttributeError("No such file: %s" % full_path)
elif not full_path.endswith(".hdf5"):
    raise AttributeError("Not a HDF5 file")

data = image_data.ImageData(full_path)

In [12]:
task = fusion.MultiViewFusionRL(data, options)

The original image size is 79 410 410
The fusion will be run with 1 blocks per dimension
The internal image size is 79 410 410


In [13]:
import cProfile

task.estimate = numpy.ones(task.image_size, dtype=numpy.float64)
cProfile.run('task.compute_estimate()')

Beginning the computation of the 0. estimate
The current block is 1
         1309 function calls in 24.797 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001   24.797   24.797 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 _methods.py:40(_all)
        6    0.000    0.000    0.000    0.000 base.py:105(_e)
       26    0.000    0.000    0.000    0.000 base.py:232(id)
        4    0.000    0.000    0.000    0.000 base.py:265(__init__)
        4    0.000    0.000    0.000    0.000 base.py:93(_lapl)
       10    0.000    0.000    0.000    0.000 dataset.py:209(shape)
        4    0.000    0.000    0.001    0.000 dataset.py:305(__init__)
        2    0.000    0.000    0.000    0.000 dataset.py:34(readtime_dtype)
        2    0.004    0.002    0.008    0.004 dataset.py:389(__getitem__)
        2    0.000    0.000    0.000    0.000 dataset.py:404(<genexpr>)
        8    0.000    0.000    

In [14]:
from supertomo.reconstruction import fusion_cuda

task2 = fusion_cuda.MultiViewFusionRLCuda(data, options)

The original image size is 79 410 410
The fusion will be run with 1 blocks per dimension
The internal image size is 79 410 410
kernel config: (13, 13, 10) x (32, 32, 8)


In [15]:
task2.estimate = numpy.ones(task2.image_size, dtype=numpy.float64)
cProfile.run('task2.compute_estimate()')

The current block is 1
         6050 function calls (5955 primitive calls) in 0.711 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.712    0.712 <string>:1(<module>)
       12    0.000    0.000    0.000    0.000 <string>:8(__new__)
       12    0.000    0.000    0.000    0.000 _internal.py:223(__init__)
       12    0.000    0.000    0.000    0.000 _internal.py:248(get_data)
       62    0.001    0.000    0.001    0.000 _methods.py:34(_prod)
        4    0.000    0.000    0.000    0.000 _methods.py:40(_all)
       72    0.000    0.000    0.000    0.000 _weakrefset.py:70(__contains__)
       36    0.001    0.000    0.001    0.000 abc.py:128(__instancecheck__)
       12    0.000    0.000    0.000    0.000 abstract.py:102(__eq__)
       12    0.000    0.000    0.001    0.000 abstract.py:42(_intern)
       12    0.000    0.000    0.003    0.000 abstract.py:54(__call__)
       12    0.000    0.00

In [16]:
task2.psfs[0].shape

(79, 410, 410)